In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
import cv2
import random
import json
from pathlib import Path
from sklearn.model_selection import train_test_split
import zipfile
import io
from PIL import Image
import shutil
from urllib.parse import urljoin

In [3]:
def create_directories(base_dir='./isic2018_data'):
    """Create necessary directories for the ISIC 2018 dataset"""
    directories = [
        base_dir,
        os.path.join(base_dir, 'task1'),
        os.path.join(base_dir, 'task1', 'images'),
        os.path.join(base_dir, 'task1', 'masks'),
        os.path.join(base_dir, 'task1', 'preprocessed'),
        os.path.join(base_dir, 'logs')
    ]
    
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"Created directory: {directory}")
    
    return base_dir